In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [8]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('./configs/trash/faster_rcnn.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 16

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn'
# cfg.work_dir = './work_dirs/yolov3_d53_320_273e_trash'

cfg.model.roi_head.bbox_head.num_classes = 11
# cfg.model.bbox_head.num_classes = 11


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [9]:
cfg.data.train.pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'LoadAnnotations', 'with_bbox': True},
 {'type': 'Resize', 'img_scale': (512, 512), 'keep_ratio': True},
 {'type': 'RandomFlip', 'flip_ratio': 0.5},
 {'type': 'Normalize',
  'mean': [123.675, 116.28, 103.53],
  'std': [58.395, 57.12, 57.375],
  'to_rgb': True},
 {'type': 'Pad', 'size_divisor': 32},
 {'type': 'DefaultFormatBundle'},
 {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}]

In [10]:
cfg.data.val.pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'MultiScaleFlipAug',
  'img_scale': (512, 512),
  'flip': False,
  'transforms': [{'type': 'Resize', 'keep_ratio': True},
   {'type': 'RandomFlip'},
   {'type': 'Normalize',
    'mean': [123.675, 116.28, 103.53],
    'std': [58.395, 57.12, 57.375],
    'to_rgb': True},
   {'type': 'Pad', 'size_divisor': 32},
   {'type': 'ImageToTensor', 'keys': ['img']},
   {'type': 'Collect', 'keys': ['img']}]}]

In [11]:
model = build_detector(cfg.model)

2021-05-15 19:05:30,088 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-15 19:05:30,092 - mmdet - INFO - Use load_from_torchvision loader
2021-05-15 19:05:30,366 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [12]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.34s)
creating index...
index created!


In [13]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-15 19:05:36,135 - mmdet - INFO - Start running, host: root@45fd1f8b8b06, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/faster_rcnn
2021-05-15 19:05:36,138 - mmdet - INFO - workflow: [('train', 1)], max: 30 epochs


Done (t=1.05s)
creating index...
index created!


/opt/ml/code/mmdetection_trash/mmdet/core/evaluation/eval_hooks.py:112: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')


train/loss_rpn_cls,0.16218
train/loss_rpn_bbox,0.08247
train/loss_cls,0.43355
train/acc,88.8689
train/loss_bbox,0.40633
train/loss,1.08453
train/grad_norm,3.6076
learning_rate,0.0003
momentum,0.9
_runtime,320
_timestamp,1621104643


...

Loading and preparing results...
DONE (t=0.89s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.68s).
Accumulating evaluation results...


2021-05-15 21:18:14,494 - mmdet - INFO - Epoch(val) [30][164]	bbox_mAP: 0.1360, bbox_mAP_50: 0.2340, bbox_mAP_75: 0.1360, bbox_mAP_s: 0.0260, bbox_mAP_m: 0.1180, bbox_mAP_l: 0.2700, bbox_mAP_copypaste: 0.136 0.234 0.136 0.026 0.118 0.270


DONE (t=0.75s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.136
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.234
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.136
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.026
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.118
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.270
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.281
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.281
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.281
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.076
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.246
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.549


train/loss_rpn_cls,0.02104
train/loss_rpn_bbox,0.0412
train/loss_cls,0.20869
train/acc,93.19751
train/loss_bbox,0.2268
train/loss,0.49774
train/grad_norm,1.2251
learning_rate,1e-05
momentum,0.9
_runtime,7955
_timestamp,1621113494


train/loss_rpn_cls,█▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▇▆▅▄▄▅▄▄▄▃▂▂▂▂▂▁▂▂▂▁▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_cls,█▅▅▄▄▄▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁▂▂▃▃▃▂▃▃▃▄▄▅▅▅▆▆▆▆▆▇▆▆▆▇▇▇▇▇▇▇▇▇█▇█████
train/loss_bbox,▂█▇▆▅▅▅▄▅▄▄▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▅▅▄▄▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,█▃▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,▂▃▅▇███████▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
_timestamp,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
